In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("/kaggle/input/quora/train.csv")

In [3]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            404290 non-null  int64 
 1   qid1          404290 non-null  int64 
 2   qid2          404290 non-null  int64 
 3   question1     404289 non-null  object
 4   question2     404288 non-null  object
 5   is_duplicate  404290 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


In [5]:
df.is_duplicate.value_counts()

is_duplicate
0    255027
1    149263
Name: count, dtype: int64

In [6]:
df.dropna(axis=0,inplace=True)

****Basic features engineering****

In [7]:
df['freq_qid1']=df.groupby('qid1')['qid1'].transform('count')
df['freq_qid2']=df.groupby('qid2')['qid2'].transform('count')
df['q1len']=df['question1'].str.len()
df['q2len']=df['question2'].str.len()
df['q1_n_words']=df['question1'].apply(lambda x:len(x.split(" ")))
df['q2_n_words']=df['question2'].apply(lambda x:len(x.split(" ")))

In [8]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,1,1,66,57,14,12
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,4,1,51,88,8,13
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,1,1,73,59,14,10
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,1,1,50,65,11,9
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,3,1,76,39,13,7


In [9]:
def words_common(row1,row2):
    a=set(row1.split(" "))
    b=set(row2.split(" "))
    return len(a&b)

def total_words(row1,row2):
    a=set(row1.split(" "))
    b=set(row2.split(" "))
    return len(a)+len(b)

df['words_common']=df.apply(lambda x:words_common(x['question1'],x['question2']),axis=1)
df['words_total']=df.apply(lambda x:total_words(x['question1'],x['question2']),axis=1)
df['words_share']=df.words_common/df.words_total

In [10]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,words_common,words_total,words_share
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,1,1,66,57,14,12,10,23,0.434783
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,4,1,51,88,8,13,4,20,0.200000
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,1,1,73,59,14,10,3,24,0.125000
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,1,1,50,65,11,9,0,19,0.000000
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,3,1,76,39,13,7,2,20,0.100000


****Advanced feature engineering****

In [12]:
CONTRACTION_MAP = {
    "ain't": "is not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

In [13]:
#defining a function to expand contracted words
def contract_expand(text):
    new_text=""
    text=text.lower()
    for word in text.split():
        if word in CONTRACTION_MAP.keys():
            new_text=new_text+CONTRACTION_MAP[word]
            new_text=new_text+" "
        else:
            new_text=new_text+word
            new_text=new_text+" "
    return new_text

In [14]:
#checking our function
sample_text="I dON't eat banana"
contract_expand(sample_text)

'i do not eat banana '

**Now we will apply this function to our dataset for qustion1 and question2**

In [15]:
df['question1']=df.question1.apply(contract_expand)
df['question2']=df.question2.apply(contract_expand)

****Removing punctuation****



In [16]:
#defining function
def remove_punctuation_and_links(text):
    pattern1=re.compile('https?\S+|www.\S+|#[a-zA-Z0-9]+')
    text=re.sub(pattern1,"",text)
    pattern2="[^\w\s]"

    return re.sub(pattern2,"",text)


In [17]:
import warnings
warnings.filterwarnings("ignore")

from matplotlib import style
style.use("ggplot")

import regex as re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer



from fuzzywuzzy import fuzz
from nltk.tokenize import word_tokenize

import spacy
from tqdm import tqdm
%matplotlib inline

In [18]:
import re
import string

def remove_punctuation_and_links(text):
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

# Sample text
sample_text = "What a shot! Sky is in absolute form this year. (sky is nickname for surya kumar yadav .for reference https://en.wikipedia.org/wiki/Suryakumar_Yadav)"

# Call the function
cleaned_text = remove_punctuation_and_links(sample_text)
print(cleaned_text)


What a shot Sky is in absolute form this year sky is nickname for surya kumar yadav for reference 


In [19]:
%%time

df['question1']=df.question1.apply(remove_punctuation_and_links)
df['question2']=df.question2.apply(remove_punctuation_and_links)

CPU times: user 3.9 s, sys: 57.1 ms, total: 3.96 s
Wall time: 3.96 s


****Word Tokenization and Stemming****

In [20]:
ps=PorterStemmer()

def stem(text):
    tokens=text.split()
    token_stemmed=[ps.stem(word) for word in tokens]
    new_text=remove_punctuation_and_links(str(token_stemmed))
    return new_text

In [21]:
#checking our stem function
a="my name is riya ,i follow arijit singh and i m followed by my juniors"

stem(a)

'my name is riya i follow arijit singh and i m follow by my junior'

Lets apply this on dataset now

In [22]:
%%time
df['question1']=df.question1.apply(stem)

CPU times: user 1min, sys: 37.8 ms, total: 1min
Wall time: 1min


In [23]:
%%time
df['question2']=df.question2.apply(stem)

CPU times: user 1min 1s, sys: 55.3 ms, total: 1min 1s
Wall time: 1min 1s


Advance Feature Creation

In [25]:
from nltk.corpus import stopwords

try:
    stop_words = set(stopwords.words('english'))
    print("Stopwords loaded successfully!")
except LookupError:
    print("Stopwords not found!")


Stopwords loaded successfully!


In [26]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
stopword = list(ENGLISH_STOP_WORDS)


In [29]:
def get_features(q1,q2):
    
    token_features=[0]*10
    
    #converting strings to tokens
    q1_tokens=q1.split()
    q2_tokens=q2.split()
    
    
    if len(q1_tokens)==0 or len(q2_tokens)==0:
        return token_features
    
    #getting words without stopwords
    q1_words=set(word for word in q1_tokens if word not in stopword)
    q2_words=set(word for word in q2_tokens if word not in stopword)
    
    #getting stopwords
    q1_sw=set(word for word in q1_tokens if word in stopword)
    q2_sw=set(word for word in q2_tokens if word in stopword)
    
    token_features[0]=len(q1_words&q2_words)/(min(len(q1_words),len(q2_words))+0.0001)
    token_features[1]=len(q1_words&q2_words)/(max(len(q1_words),len(q2_words))+0.0001)
    token_features[2]=len(q1_sw&q2_sw)/(min(len(q1_sw),len(q2_sw))+0.0001)   #for safe division
    token_features[3]=len(q1_sw&q2_sw)/(max(len(q1_sw),len(q2_sw))+0.0001)
    token_features[4]=len(set(q1_tokens)&set(q2_tokens))/(min(len(q1_tokens),len(q2_tokens))+0.0001)
    token_features[5]=len(set(q1_tokens)&set(q2_tokens))/(max(len(q1_tokens),len(q2_tokens))+0.0001)
    
    token_features[6]=int(q1_tokens[-1]==q2_tokens[-1])
    token_features[7]=int(q1_tokens[0]==q2_tokens[0])
    token_features[8]=len(q1_tokens)-len(q2_tokens)
    token_features[9]=(len(q1_tokens)+len(q2_tokens))/2
    
    
    return token_features
    
    
    
    

In [30]:
%%time

print("Creating features....")
df['cwc_min']=df.apply(lambda x:get_features(x['question1'],x['question2'])[0],axis=1)
df['cwc_max']=df.apply(lambda x:get_features(x['question1'],x['question2'])[1],axis=1)
df['csc_min']=df.apply(lambda x:get_features(x['question1'],x['question2'])[2],axis=1)
df['csc_max']=df.apply(lambda x:get_features(x['question1'],x['question2'])[3],axis=1)
df['ctc_min']=df.apply(lambda x:get_features(x['question1'],x['question2'])[4],axis=1)
df['ctc_max']=df.apply(lambda x:get_features(x['question1'],x['question2'])[5],axis=1)
df['last_word_eq']=df.apply(lambda x:get_features(x['question1'],x['question2'])[6],axis=1)
df['first_word_eq']=df.apply(lambda x:get_features(x['question1'],x['question2'])[7],axis=1)
df['abs_diff']=df.apply(lambda x:get_features(x['question1'],x['question2'])[8],axis=1)
df['mean_length']=df.apply(lambda x:get_features(x['question1'],x['question2'])[9],axis=1)

Creating features....
CPU times: user 10min 2s, sys: 1.59 s, total: 10min 4s
Wall time: 10min 4s


In [31]:
pd.set_option("display.max_columns",None)
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,words_common,words_total,words_share,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_diff,mean_length
0,0,1,2,what is the step by step guid to invest in sha...,what is the step by step guid to invest in sha...,0,1,1,66,57,14,12,10,23,0.434783,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0,1,2,13.0
1,1,3,4,what is the stori of kohinoor kohinoor diamond,what would happen if the indian govern stole t...,0,4,1,51,88,8,13,4,20,0.200000,0.666644,0.333328,0.499988,0.399992,0.499994,0.307690,0,1,-5,10.5
2,2,5,6,how can i increas the speed of my internet con...,how can internet speed be increas by hack thro...,0,1,1,73,59,14,10,3,24,0.125000,0.599988,0.499992,0.399992,0.249997,0.499995,0.357140,0,1,4,12.0
3,3,7,8,whi am i mental veri lone how can i solv it,find the remaind when math2324math is divid by...,0,1,1,50,65,11,9,0,19,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,2,10.0
4,4,9,10,which one dissolv in water quikli sugar salt m...,which fish would surviv in salt water,0,3,1,76,39,13,7,2,20,0.100000,0.499988,0.222220,0.666644,0.499988,0.571420,0.307690,0,1,6,10.0


In [32]:
#creatuing fuzzy features
print("Creating fuzzy feautures .....")

df['fuzz_ratio']=df.apply(lambda x:fuzz.QRatio(x['question1'],x['question2']),axis=1)
df['fuzz_partial_ratio']=df.apply(lambda x:fuzz.partial_ratio(x['question1'],x['question2']),axis=1)
df['token_sort_ratio']=df.apply(lambda x:fuzz.token_sort_ratio(x['question1'],x['question2']),axis=1)
df['token_set_ratio']=df.apply(lambda x:fuzz.token_set_ratio(x['question1'],x['question2']),axis=1)

Creating fuzzy feautures .....


In [33]:
print("The shape of the data is",df.shape)
print(df.columns)

The shape of the data is (404287, 29)
Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate',
       'freq_qid1', 'freq_qid2', 'q1len', 'q2len', 'q1_n_words', 'q2_n_words',
       'words_common', 'words_total', 'words_share', 'cwc_min', 'cwc_max',
       'csc_min', 'csc_max', 'ctc_min', 'ctc_max', 'last_word_eq',
       'first_word_eq', 'abs_diff', 'mean_length', 'fuzz_ratio',
       'fuzz_partial_ratio', 'token_sort_ratio', 'token_set_ratio'],
      dtype='object')


In [34]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# Combine question1 and question2 into a single text field
df['text'] = df['question1'].fillna('') + ' ' + df['question2'].fillna('')


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
df['tokens'] = df['text'].apply(lambda x: word_tokenize(str(x).lower()))


In [36]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(sentences=df['tokens'], vector_size=100, window=5, min_count=1, workers=4)


In [37]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['tokens'])
sequences = tokenizer.texts_to_sequences(df['tokens'])
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

maxlen = 100
X = pad_sequences(sequences, maxlen=maxlen)


In [38]:
import numpy as np

embedding_dim = 100
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]


In [39]:
from sklearn.model_selection import train_test_split

y = df['is_duplicate'].values  # This is the label column
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

model_rnn = Sequential()
model_rnn.add(Embedding(input_dim=vocab_size,
                        output_dim=embedding_dim,
                        weights=[embedding_matrix],
                        input_length=maxlen,
                        trainable=False))
model_rnn.add(SimpleRNN(64))
model_rnn.add(Dense(1, activation='sigmoid'))

model_rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_rnn.summary()

# Train
history_rnn = model_rnn.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate
loss_rnn, acc_rnn = model_rnn.evaluate(X_test, y_test)
print(f"🧠 RNN Test Accuracy: {acc_rnn:.4f}")


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │       8,696,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,696,100 (33.17 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 8,696,100 (33.17 MB)

Epoch 1/10
10108/10108 ━━━━━━━━━━━━━━━━━━━━ 105s 10ms/step - accuracy: 0.6914 - loss: 0.5958 - val_accuracy: 0.7114 - val_loss: 0.5697
Epoch 2/10
10108/10108 ━━━━━━━━━━━━━━━━━━━━ 99s 10ms/step - accuracy: 0.7308 - loss: 0.5475 - val_accuracy: 0.7335 - val_loss: 0.5380
Epoch 3/10
10108/10108 ━━━━━━━━━━━━━━━━━━━━ 100s 10ms/step - accuracy: 0.7292 - loss: 0.5488 - val_accuracy: 0.7287 - val_loss: 0.5404
Epoch 4/10
10108/10108 ━━━━━━━━━━━━━━━━━━━━ 99s 10ms/step - accuracy: 0.7361 - loss: 0.5390 - val_accuracy: 0.7279 - val_loss: 0.5467
Epoch 5/10
10108/10108 ━━━━━━━━━━━━━━━━━━━━ 100s 10ms/step - accuracy: 0.7374 - loss: 0.5379 - val_accuracy: 0.7370 - val_loss: 0.5355
Epoch 6/10
10108/10108 ━━━━━━━━━━━━━━━━━━━━ 100s 10ms/step - accuracy: 0.7367 - loss: 0.5383 - val_accuracy: 0.7367 - val_loss: 0.5361
Epoch 7/10
10108/10108 ━━━━━━━━━━━━━━━━━━━━ 99s 10ms/step - accuracy: 0.7413 - loss: 0.5317 - val_accuracy: 0.7399 - val_loss: 0.5325
Epoch 8/10
10108/10108 ━━━━━━━━━━━━━━━━━━━━ 100s 10ms/step

In [41]:
model_rnn.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 100, 100)            │       8,696,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ (None, 64)                  │          10,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,727,977 (33.29 MB)

 Trainable params: 10,625 (41.50 KB)

 Non-trainable params: 8,696,100 (33.17 MB)

 Optimizer params: 21,252 (83.02 KB)

In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dropout, Dense, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Step 1: Compute class weights to handle imbalance
class_weights_array = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights_array))

# Step 2: Define the model
model_rnn = Sequential()
model_rnn.add(Embedding(input_dim=vocab_size,
                        output_dim=embedding_dim,
                        weights=[embedding_matrix],
                        input_length=maxlen,
                        trainable=False))  # Freeze embedding layer

# Add Bidirectional RNN
model_rnn.add(Bidirectional(SimpleRNN(64)))

# Dropout for regularization
model_rnn.add(Dropout(0.5))

# Output layer
model_rnn.add(Dense(1, activation='sigmoid'))

# Step 3: Compile the model
optimizer = Adam(learning_rate=0.001)
model_rnn.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Step 4: Print model summary
model_rnn.summary()

# Step 5: Train the model
history_rnn = model_rnn.fit(
    X_train, y_train,
    epochs=10,
    batch_size=64,
    validation_data=(X_test, y_test),
    class_weight=class_weights,
    verbose=1
)

# Step 6: Evaluate performance
loss_rnn, acc_rnn = model_rnn.evaluate(X_test, y_test, verbose=0)
print(f"\n📈 Improved RNN Test Accuracy: {acc_rnn:.4f}") 

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │       8,696,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,696,100 (33.17 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 8,696,100 (33.17 MB)

Epoch 1/10
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 104s 20ms/step - accuracy: 0.6486 - loss: 0.6421 - val_accuracy: 0.6924 - val_loss: 0.5977
Epoch 2/10
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 98s 19ms/step - accuracy: 0.7035 - loss: 0.5869 - val_accuracy: 0.7216 - val_loss: 0.5605
Epoch 3/10
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 98s 19ms/step - accuracy: 0.7136 - loss: 0.5756 - val_accuracy: 0.7085 - val_loss: 0.5849
Epoch 4/10
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 97s 19ms/step - accuracy: 0.7172 - loss: 0.5705 - val_accuracy: 0.7341 - val_loss: 0.5454
Epoch 5/10
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 98s 19ms/step - accuracy: 0.7200 - loss: 0.5669 - val_accuracy: 0.7386 - val_loss: 0.5378
Epoch 6/10
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 98s 19ms/step - accuracy: 0.7189 - loss: 0.5646 - val_accuracy: 0.7117 - val_loss: 0.5692
Epoch 7/10
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 98s 19ms/step - accuracy: 0.7226 - loss: 0.5643 - val_accuracy: 0.7032 - val_loss: 0.5825
Epoch 8/10
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 98s 19ms/step - accuracy: 0.7214 

In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.optimizers import Adam
import numpy as np

# Compute class weights
class_weights_array = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights_array))

# Define LSTM model
model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=vocab_size,
                         output_dim=embedding_dim,
                         weights=[embedding_matrix],
                         input_length=maxlen,
                         trainable=False))
model_lstm.add(LSTM(128))
model_lstm.add(Dropout(0.5))
model_lstm.add(Dense(1, activation='sigmoid'))

# Compile model
model_lstm.compile(optimizer=Adam(learning_rate=0.001),
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

# Summary
model_lstm.summary()

# Train
history_lstm = model_lstm.fit(X_train, y_train,
                              epochs=30,
                              batch_size=64,
                              validation_data=(X_test, y_test),
                              class_weight=class_weights,
                              verbose=1)

# Evaluate
loss_lstm, acc_lstm = model_lstm.evaluate(X_test, y_test, verbose=0)
print(f"\n🔁 LSTM Test Accuracy: {acc_lstm:.4f}")


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │       8,696,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,696,100 (33.17 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 8,696,100 (33.17 MB)

Epoch 1/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 40s 7ms/step - accuracy: 0.7064 - loss: 0.5654 - val_accuracy: 0.7570 - val_loss: 0.4882
Epoch 2/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 36s 7ms/step - accuracy: 0.7556 - loss: 0.4917 - val_accuracy: 0.7716 - val_loss: 0.4634
Epoch 3/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 36s 7ms/step - accuracy: 0.7761 - loss: 0.4571 - val_accuracy: 0.7649 - val_loss: 0.4733
Epoch 4/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 36s 7ms/step - accuracy: 0.7900 - loss: 0.4328 - val_accuracy: 0.7798 - val_loss: 0.4502
Epoch 5/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 36s 7ms/step - accuracy: 0.8031 - loss: 0.4108 - val_accuracy: 0.7794 - val_loss: 0.4545
Epoch 6/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 36s 7ms/step - accuracy: 0.8117 - loss: 0.3937 - val_accuracy: 0.7866 - val_loss: 0.4444
Epoch 7/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 36s 7ms/step - accuracy: 0.8210 - loss: 0.3776 - val_accuracy: 0.7863 - val_loss: 0.4454
Epoch 8/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 36s 7ms/step - accuracy: 0.8282 - loss: 0

In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=vocab_size,
                         output_dim=embedding_dim,
                         weights=[embedding_matrix],
                         input_length=maxlen,
                         trainable=False))
model_lstm.add(LSTM(64))
model_lstm.add(Dense(1, activation='sigmoid'))

model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_lstm.summary()

# Train
history_lstm = model_lstm.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))

# Evaluate
loss, accuracy = model_lstm.evaluate(X_test, y_test)
print(f"LSTM Test Accuracy: {accuracy:.4f}")


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │       8,696,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,696,100 (33.17 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 8,696,100 (33.17 MB)

Epoch 1/30
10108/10108 ━━━━━━━━━━━━━━━━━━━━ 65s 6ms/step - accuracy: 0.7358 - loss: 0.5274 - val_accuracy: 0.7681 - val_loss: 0.4744
Epoch 2/30
10108/10108 ━━━━━━━━━━━━━━━━━━━━ 64s 6ms/step - accuracy: 0.7780 - loss: 0.4611 - val_accuracy: 0.7760 - val_loss: 0.4604
Epoch 3/30
10108/10108 ━━━━━━━━━━━━━━━━━━━━ 64s 6ms/step - accuracy: 0.7933 - loss: 0.4346 - val_accuracy: 0.7827 - val_loss: 0.4496
Epoch 4/30
10108/10108 ━━━━━━━━━━━━━━━━━━━━ 64s 6ms/step - accuracy: 0.8040 - loss: 0.4164 - val_accuracy: 0.7859 - val_loss: 0.4464
Epoch 5/30
10108/10108 ━━━━━━━━━━━━━━━━━━━━ 64s 6ms/step - accuracy: 0.8147 - loss: 0.4007 - val_accuracy: 0.7869 - val_loss: 0.4450
Epoch 6/30
10108/10108 ━━━━━━━━━━━━━━━━━━━━ 64s 6ms/step - accuracy: 0.8197 - loss: 0.3914 - val_accuracy: 0.7889 - val_loss: 0.4451
Epoch 7/30
10108/10108 ━━━━━━━━━━━━━━━━━━━━ 64s 6ms/step - accuracy: 0.8264 - loss: 0.3787 - val_accuracy: 0.7900 - val_loss: 0.4438
Epoch 8/30
10108/10108 ━━━━━━━━━━━━━━━━━━━━ 64s 6ms/step - accuracy: 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.optimizers import Adam
import numpy as np

# Compute class weights
class_weights_array = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights_array))

# Define LSTM model
model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=vocab_size,
                         output_dim=embedding_dim,
                         weights=[embedding_matrix],
                         input_length=maxlen,
                         trainable=False))
model_lstm.add(LSTM(128))
model_lstm.add(Dropout(0.5))
model_lstm.add(Dense(1, activation='sigmoid'))

# Compile model
model_lstm.compile(optimizer=Adam(learning_rate=0.001),
                   loss='binary_crossentropy',
                   metrics=['accuracy'])



# Train
history_lstm = model_lstm.fit(X_train, y_train,
                              epochs=20,
                              batch_size=64,
                              validation_data=(X_test, y_test),
                              class_weight=class_weights,
                              verbose=1)

# Evaluate
loss_lstm, acc_lstm = model_lstm.evaluate(X_test, y_test, verbose=0)
print(f"\n🔁 LSTM Test Accuracy: {acc_lstm:.4f}")


In [ ]:
model_lstm.summary()


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.optimizers import Adam
import numpy as np

# Compute class weights
class_weights_array = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights_array))

# Define LSTM model
model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=vocab_size,
                         output_dim=embedding_dim,
                         weights=[embedding_matrix],
                         input_length=maxlen,
                         trainable=False))
model_lstm.add(LSTM(128))
model_lstm.add(Dropout(0.5))
model_lstm.add(Dense(1, activation='sigmoid'))

# Compile model
model_lstm.compile(optimizer=Adam(learning_rate=0.001),
                   loss='binary_crossentropy',
                   metrics=['accuracy'])



# Train
history_lstm = model_lstm.fit(X_train, y_train,
                              epochs=30,
                              batch_size=64,
                              validation_data=(X_test, y_test),
                              class_weight=class_weights,
                              verbose=1)

# Evaluate
loss_lstm, acc_lstm = model_lstm.evaluate(X_test, y_test, verbose=0)
print(f"\n🔁 LSTM Test Accuracy: {acc_lstm:.4f}")


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Compute class weights
class_weights_array = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights_array))

# Define the model
model = Sequential()
model.add(Embedding(input_dim=vocab_size,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=maxlen,
                    trainable=False))  # Use pre-trained embeddings (Word2Vec or GloVe)

# Bidirectional LSTM with L2 regularization
model.add(Bidirectional(LSTM(128, kernel_regularizer=l2(0.001))))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Model callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True)



# Train the model
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=64,
    validation_data=(X_test, y_test),
    class_weight=class_weights,
    callbacks=[early_stop, checkpoint],
    verbose=1
)



In [ ]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Final BiLSTM Test Accuracy: {acc:.4f}")


In [ ]:
print("hi")

In [46]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Compute class weights
class_weights_array = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights_array))


model = Sequential()
model.add(Embedding(input_dim=vocab_size,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=maxlen,
                    trainable=True))  # <-- trainable now

model.add(SpatialDropout1D(0.3))  # better dropout

model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Model callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True)



# Train the model
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=64,
    validation_data=(X_test, y_test),
    class_weight=class_weights,
    callbacks=[early_stop, checkpoint],
    verbose=1
)



Epoch 1/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 142s 27ms/step - accuracy: 0.6955 - loss: 0.5771 - val_accuracy: 0.7576 - val_loss: 0.4851
Epoch 2/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 138s 27ms/step - accuracy: 0.7612 - loss: 0.4844 - val_accuracy: 0.7825 - val_loss: 0.4455
Epoch 3/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 138s 27ms/step - accuracy: 0.7937 - loss: 0.4263 - val_accuracy: 0.7954 - val_loss: 0.4271
Epoch 4/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 138s 27ms/step - accuracy: 0.8191 - loss: 0.3789 - val_accuracy: 0.7962 - val_loss: 0.4394
Epoch 5/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 138s 27ms/step - accuracy: 0.8357 - loss: 0.3452 - val_accuracy: 0.8004 - val_loss: 0.4396
Epoch 6/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 138s 27ms/step - accuracy: 0.8461 - loss: 0.3201 - val_accuracy: 0.8002 - val_loss: 0.4468


In [47]:
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K

class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="glorot_uniform", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1),
                                 initializer="zeros", trainable=True)
        super(Attention, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        e = K.squeeze(e, axis=-1)
        alpha = K.softmax(e)
        alpha = K.expand_dims(alpha, axis=-1)
        context = x * alpha
        context = K.sum(context, axis=1)
        return context


In [48]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dropout, Dense, Bidirectional, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Input layer
input_layer = Input(shape=(maxlen,))
embedding_layer = Embedding(input_dim=vocab_size,
                            output_dim=embedding_dim,
                            weights=[embedding_matrix],
                            input_length=maxlen,
                            trainable=True)(input_layer)

x = SpatialDropout1D(0.3)(embedding_layer)

x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Bidirectional(LSTM(64, return_sequences=True))(x)

# Attention
attention_output = Attention()(x)

x = Dropout(0.5)(attention_output)
output_layer = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile
optimizer = Adam(learning_rate=1e-4)  # slightly lower LR
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model_with_attention.keras', monitor='val_accuracy', save_best_only=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)

# Train
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=64,
    validation_data=(X_test, y_test),
    class_weight=class_weights,
    callbacks=[early_stop, checkpoint, lr_reduce],
    verbose=1
)


Epoch 1/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 145s 28ms/step - accuracy: 0.6662 - loss: 0.6048 - val_accuracy: 0.7372 - val_loss: 0.5228 - learning_rate: 1.0000e-04
Epoch 2/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 141s 28ms/step - accuracy: 0.7201 - loss: 0.5508 - val_accuracy: 0.7543 - val_loss: 0.5032 - learning_rate: 1.0000e-04
Epoch 3/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 141s 28ms/step - accuracy: 0.7311 - loss: 0.5330 - val_accuracy: 0.7627 - val_loss: 0.4875 - learning_rate: 1.0000e-04
Epoch 4/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 141s 28ms/step - accuracy: 0.7414 - loss: 0.5173 - val_accuracy: 0.7642 - val_loss: 0.4808 - learning_rate: 1.0000e-04
Epoch 5/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 141s 28ms/step - accuracy: 0.7486 - loss: 0.5043 - val_accuracy: 0.7657 - val_loss: 0.4765 - learning_rate: 1.0000e-04
Epoch 6/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 141s 28ms/step - accuracy: 0.7547 - loss: 0.4934 - val_accuracy: 0.7744 - val_loss: 0.4618 - learning_rate: 1.0000e-04
Epoch 7/30
5054/5054 ━━━━━━━━━━━━━

In [50]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dropout, Dense, Bidirectional, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
class_weights_array = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights_array))

# Attention Layer
class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="glorot_uniform", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1),
                                 initializer="zeros", trainable=True)
        super(Attention, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        e = K.squeeze(e, axis=-1)
        alpha = K.softmax(e)
        alpha = K.expand_dims(alpha, axis=-1)
        context = x * alpha
        context = K.sum(context, axis=1)
        return context

# Model architecture
input_layer = Input(shape=(maxlen,))
embedding_layer = Embedding(input_dim=vocab_size,
                            output_dim=embedding_dim,
                            weights=[embedding_matrix],
                            input_length=maxlen,
                            trainable=True)(input_layer)

x = SpatialDropout1D(0.3)(embedding_layer)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Bidirectional(LSTM(64, return_sequences=True))(x)
x = Attention()(x)
x = Dropout(0.5)(x)
output_layer = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile model
optimizer = Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
checkpoint = ModelCheckpoint('best_model_with_attention.keras', monitor='val_accuracy', save_best_only=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)

# Train model (no EarlyStopping)
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=64,
    validation_data=(X_test, y_test),
    class_weight=class_weights,
    callbacks=[checkpoint, lr_reduce],
    verbose=1
)


Epoch 1/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 145s 28ms/step - accuracy: 0.6661 - loss: 0.6054 - val_accuracy: 0.7434 - val_loss: 0.5157 - learning_rate: 1.0000e-04
Epoch 2/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 141s 28ms/step - accuracy: 0.7248 - loss: 0.5479 - val_accuracy: 0.7537 - val_loss: 0.5005 - learning_rate: 1.0000e-04
Epoch 3/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 142s 28ms/step - accuracy: 0.7347 - loss: 0.5296 - val_accuracy: 0.7603 - val_loss: 0.4858 - learning_rate: 1.0000e-04
Epoch 4/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 142s 28ms/step - accuracy: 0.7413 - loss: 0.5140 - val_accuracy: 0.7663 - val_loss: 0.4737 - learning_rate: 1.0000e-04
Epoch 5/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 142s 28ms/step - accuracy: 0.7472 - loss: 0.5049 - val_accuracy: 0.7715 - val_loss: 0.4659 - learning_rate: 1.0000e-04
Epoch 6/30
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 142s 28ms/step - accuracy: 0.7559 - loss: 0.4917 - val_accuracy: 0.7757 - val_loss: 0.4577 - learning_rate: 1.0000e-04
Epoch 7/30
5054/5054 ━━━━━━━━━━━━━